In [3]:
import numpy as np
import pandas as pd
import kmapper as km
import kneed
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from networkx.readwrite import json_graph
import json
import collections
from dataclasses import dataclass
import ast
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from diskcache import Cache

sns.set()
cache = Cache('./cache/')

In [4]:
NUM_EPOCHS = 176
# NUM_POINTS = 457

NUM_POINTS = 4282

LAYER = 12
DATA = 'train'

In [5]:
@dataclass
class Config:
    metric: str = 'euclidean'
    filter_func: str = 'l2'
    intervals: int = 50
    overlap: float = 0.5


def read_labels(path):
    label_data = []
    with open(path, 'r') as f:
        for line in f:
            word_info, word_label = line.strip().split('\t')
            sent_info, word = word_info.split(':')
            sent_info = ast.literal_eval(sent_info)
            label_data.append([sent_info[0], sent_info[1], word, word_label])

    return pd.DataFrame(label_data, columns=['sent_id', 'word_id', 'word', 'label'])


def add_node_metadata(graph, metadata_source, activations):
    # create PCA model first
    nodewise_activations = np.vstack([np.mean(activations.iloc[graph['nodes'][node_name]], axis=0) for node_name in graph['nodes']])

    for i, node_name in enumerate(graph['nodes']):
        member_list = graph['nodes'][node_name]

        metadata = [metadata_source.loc[member_index].tolist() for member_index in member_list]
        graph['nodes'][node_name] = {'membership_ids': member_list, 'metadata': metadata,
                                     'l2avg': np.average(metadata_source.loc[member_list]['l2norm']),
                                     'type': 'train'}

    return graph


def elbow_eps(data):
    nbrs = NearestNeighbors(n_neighbors=2).fit(data)
    distances, indices = nbrs.kneighbors(data)
    distances = np.sort(distances, axis=0)[::-1]
    kneedle = kneed.KneeLocator(distances[:, 1], np.linspace(0, 1, num=len(distances)), curve='convex', direction='decreasing')
    eps = kneedle.knee * 0.75 if kneedle.knee else 5
    return eps


def serialize_graph(graph):
    nx_graph = km.adapter.to_networkx(graph)
    js_graph = json_graph.node_link_data(nx_graph)

    for i, node in enumerate(js_graph['nodes']):
        js_graph['nodes'][i]['name'] = js_graph['nodes'][i]['id']
        js_graph['nodes'][i]['l2avg'] = js_graph['nodes'][i]['membership']['l2avg']

    for i, link in enumerate(js_graph['links']):
        id_s = link['source']
        id_t = link['target']
        mem1 = [x['membership']['membership_ids'] for x in js_graph['nodes'] if x['id'] == id_s][0]
        mem2 = [x['membership']['membership_ids'] for x in js_graph['nodes'] if x['id'] == id_t][0]
        mem1, mem2 = set(mem1), set(mem2)
        jaccard = len(mem1.intersection(mem2)) / len(mem1.union(mem2))
        js_graph['links'][i]['intersection'] = jaccard

    return js_graph


def get_mapper(activations, labels, conf):
    labels['l2norm'] = np.expand_dims(np.linalg.norm(activations.to_numpy(), axis=1), 1)
    mapper = km.KeplerMapper()

    if conf.filter_func == 'l1':
        projected_data = np.linalg.norm(activations, ord=1, axis=1).reshape((activations.shape[0], 1))
    elif conf.filter_func == 'l2':
        projected_data = mapper.fit_transform(activations, projection='l2norm')
    elif conf.filter_func == 'knn5':
        projected_data = mapper.fit_transform(activations, projection='knn_distance_5') / 5
    else:
        raise KeyError('Unexpected filter function')

    eps = elbow_eps(activations)
    graph = mapper.map(projected_data, activations, clusterer=DBSCAN(eps=eps, metric=conf.metric, min_samples=5),
                       cover=km.Cover(n_cubes=conf.intervals, perc_overlap=conf.overlap))

    add_node_metadata(graph, labels, activations)

    return serialize_graph(graph)


def node_purity(node, label):
    metadata = node['membership']['metadata']
    label_counts = collections.Counter([x[3] for x in metadata])
    return (label_counts[label], len(metadata), label)


def compute_purities(graph):
    point_node_purities = collections.defaultdict(list)

    for node in graph['nodes']:
        metadata = node['membership']['metadata']

        for i, point_id in enumerate(node['membership']['membership_ids']):
            point_node_purities[point_id].append((i, node_purity(node, metadata[i][3])))

    return point_node_purities


def get_activations_labels(activation_file, label_file):
    activations = pd.read_csv(activation_file, delim_whitespace=True, header=None)
    labels = read_labels(label_file)
    return activations, labels


def wrapper(iteration, caching=True):
    layer = LAYER

    ACTIVATION_FILE = f'../backend/data/ss-role/fine-tuned-bert-base-uncased/{DATA}/{iteration}/{layer}.txt'
    label_file = f'../backend/data/ss-role/entities/{DATA}.txt'

    activations, labels = get_activations_labels(ACTIVATION_FILE, label_file)
    config = Config(metric='euclidean', filter_func='l2', intervals=50, overlap=0.5)

    cache_key = f'{ACTIVATION_FILE}-{label_file}-{config}'

    if caching and cache_key in cache:
        graph = cache[cache_key]
    else:
        graph = get_mapper(activations, labels, config)
        cache[cache_key] = graph

    return compute_purities(graph)


In [6]:
# %%time
# purities = [compute_purities(fileindex) for fileindex in tqdm(range(NUM_EPOCHS))]
# purities = Parallel(n_jobs=5)(delayed(wrapper)(fileindex) for fileindex in tqdm(range(0, NUM_EPOCHS, 5)))

purities = []

for fileindex in tqdm(range(0, NUM_EPOCHS, 5)):
    purities.append(wrapper(fileindex))


  0%|          | 0/36 [00:00<?, ?it/s]

In [7]:
def purity_df(purities):
    def purity_summary(purities):
        if purities is None:
            return -1
        else:
            plist = [x[1] for x in purities]

            numerator = sum([x[0] for x in plist])
            denominator = sum([x[1] for x in plist])
            if denominator != 0:
                return numerator / denominator
            else:
                return -1

    point_purity_tracker = collections.defaultdict(list)

    for point_number in range(NUM_POINTS):
        for iteration in range(len(range(0, NUM_EPOCHS, 5))):
            iteration_purity_dict = purities[iteration]

            point_purity_tracker[point_number].append((iteration, purity_summary(iteration_purity_dict[point_number])))

    node_purities_df = pd.DataFrame(columns=['pointNum'] + [f'epoch_{x}_purity' for x in range(0, NUM_EPOCHS, 5)])

    for pointNum in range(NUM_POINTS):
        row = [pointNum] + [x[1] for x in point_purity_tracker[pointNum]]
        node_purities_df.loc[pointNum] = row

    return node_purities_df


purity_df = purity_df(purities)


In [8]:
labels = read_labels(f'../backend/data/ss-role/entities/{DATA}.txt')
point_to_label_dict = {point_num: info['label'] for point_num, info in labels.to_dict(orient='index').items()}
purity_df = purity_df.join(labels)


In [10]:
sns.load_dataset('tips')

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [9]:
purity_df()

,pointNum,epoch_0_purity,epoch_5_purity,epoch_10_purity,epoch_15_purity,epoch_20_purity,epoch_25_purity,epoch_30_purity,epoch_35_purity,epoch_40_purity,...,epoch_150_purity,epoch_155_purity,epoch_160_purity,epoch_165_purity,epoch_170_purity,epoch_175_purity,sent_id,word_id,word,label
0,0.0,-1.000000,0.571429,0.578125,0.534884,0.500000,-1.000000,-1.000000,0.027119,-1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1,6,for,p.Purpose
1,1.0,-1.000000,0.256513,0.212897,0.106667,0.048837,0.187665,0.163114,0.369906,0.387164,...,0.992593,1.000000,1.000000,1.000000,1.000000,1.000000,2,1,My,p.SocialRel
2,2.0,-1.000000,0.196429,0.296296,0.009558,0.017551,0.045802,0.038540,0.036364,0.063462,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3,6,for,p.Cost
3,3.0,-1.000000,-1.000000,-1.000000,0.359223,0.051092,0.721649,0.643411,0.511029,0.334884,...,0.995918,0.995475,1.000000,1.000000,1.000000,1.000000,5,6,of,p.Topic
4,4.0,0.111111,0.056159,0.069191,0.057692,0.031894,0.046229,0.061567,0.145833,0.131148,...,0.106227,0.960784,0.977778,0.958333,0.959184,0.960784,6,4,my,p.Originator
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4277,4277.0,-1.000000,0.800000,-1.000000,0.149171,0.127873,0.684597,0.689300,0.682785,0.652985,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2720,19,for,p.Purpose
4278,4278.0,-1.000000,0.150000,-1.000000,-1.000000,0.336798,-1.000000,-1.000000,-1.000000,-1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2721,4,to,p.Locus
4279,4279.0,-1.000000,0.031250,-1.000000,0.031063,0.083333,0.111111,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2721,7,for,p.Direction
4280,4280.0,-1.000000,0.112000,0.110333,0.096677,0.096197,0.152500,0.208661,0.225182,0.130693,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2722,4,our,p.Agent


In [59]:
# draw ridge plot for each epoch_purity
df = purity_df.melt(id_vars='pointNum', value_vars=[f'epoch_{x}_purity' for x in range(0, NUM_EPOCHS, 5)])
df.rename(columns={'variable': 'g', 'value': 'x'}, inplace=True)
df = pd.merge(df, purity_df[['pointNum', 'label']], on='pointNum', how='left')

In [54]:
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import LabelEncoder

# epoch_105 = pd.read_csv('../backend/data/ss-role/fine-tuned-bert-base-uncased/train/105/1.txt', delim_whitespace=True, header=None)
# labels = read_labels('../backend/data/ss-role/entities/train.txt')
# label_names = labels.label
# labelEncoder = LabelEncoder().fit(label_names)

# # Draw a PCA plot of raw activations and color by labels
# pca = PCA(n_components=2)
# pca_data = pca.fit_transform(epoch_105)

# plt.figure(figsize=(15, 15))
# sns.scatterplot(x=pca_data[:, 0], y=pca_data[:, 1], hue=labels.label)

In [2]:
df.query('label in ["p.Topic"]').shape

NameError: name 'df' is not defined

In [72]:
df_filtered = df.query('label in ["p.Theme", "p.Topic"]')
sns.set_theme(style='white', rc={'axes.facecolor': (0, 0, 0, 0)})
# pal = sns.cubehelix_palette(10, rot=-.25, light=.7)
pal = sns.color_palette("crest")
# Initialize the FacetGrid object
g = sns.FacetGrid(df_filtered, row='g', col='label', hue='g', aspect=8, height=1, palette=pal)
# Draw the densities in a few steps
g.map_dataframe(sns.kdeplot, x='x', fill=True, alpha=0.75)
g.map_dataframe(sns.kdeplot, x='x', color="black")

# passing color=None to refline() uses the hue mapping
g.refline(y=0, linewidth=2, linestyle="-", color=None, clip_on=False)


# Define and use a simple function to label the plot in axes coordinates
def label(x, color, label):
    label = label.split("_")[1]
    ax = plt.gca()
    ax.text(0, .2, label, fontweight="bold", color=color, ha="left", va="center", transform=ax.transAxes)


g.map(label, "x")

# Set the subplots to overlap
g.figure.subplots_adjust(hspace=-0.75)

# Remove axes details that don't play well with overlap
g.set_titles("")
g.set(yticks=[], ylabel="")
g.despine(bottom=True, left=True)
plt.suptitle(f'Purity ridge plot, Layer={LAYER}')

plt.savefig(f'images/{LAYER}_train.jpg')

C:\Users\archi\anaconda3\envs\probing\lib\site-packages\seaborn\distributions.py:316: UserWarning: Dataset has 0 variance; skipping density estimate. Pass `warn_singular=False` to disable this warning.
  warnings.warn(msg, UserWarning)
C:\Users\archi\anaconda3\envs\probing\lib\site-packages\seaborn\distributions.py:316: UserWarning: Dataset has 0 variance; skipping density estimate. Pass `warn_singular=False` to disable this warning.
  warnings.warn(msg, UserWarning)


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x000001ED5592F948> (for post_execute):


KeyboardInterrupt: 